In [1]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np


import settings

In [2]:
start_comm_id = 21025000
comm_N = 175000

In [3]:
vk_func_getMembQuan = VkFunction(args=('x'), code='''
    var memb = [];
    var i = 0;
    while (i < 25) 
    {
        memb.push(API.groups.getMembers({"group_id": i + %(x)s})["count"]);
        i = i + 1;
    };
    return memb;
''')

vk_func_getMembIDList = VkFunction(args=('x', 'y'), code='''
    var offset = %(y)s;
    var IDList = [];
    var i = 0;
    while (i < 25)
    {
        IDList.push(API.groups.getMembers({"group_id": %(x)s, "offset" : offset + i * 1000})["items"]);
        i = i + 1;
    }
    return IDList;
''')

def isCommunityAlive(comm_id):
    try:
        info = vk.wall.get(owner_id=-comm_id, owners_only=1, count=1)
        last_post_date = datetime.datetime.fromtimestamp(info.get('items')[0].get('date'))
        last_count = datetime.datetime.now() - last_post_date
        if (last_count.days < 30):
            return "Alive"#f"Alive, {last_count.days} refresh old"
        else:
            return "Dead"#f"Dead {last_count.days} refresh old"
    except:
        return "No access"

In [4]:
vk_session = vk_api.VkApi(token=settings.MY_COMM_TOKEN) 
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
    
vk_session.RPS_DELAY = 0.05
vk = vk_session.get_api()

MQData = []
for comm_id in range(start_comm_id, start_comm_id + comm_N, 25):
    MQData.append(vk_func_getMembQuan(vk, comm_id))
    if (comm_id % 1000 == 0):
        print(comm_id - start_comm_id)

Login is required to auth
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
15

In [5]:
MQData = [y for x in MQData for y in x]
print(MQData)

[0, 173, 0, 2, 12, 1, 8, 1, 2, 1, 12, 0, 0, 213, 32, 2, 20, 18, 11, 0, 620, 9, 7, None, 199, 2, 0, 3, 5, 6, 18, None, 49, 9, 770, 0, 5, None, 11, 0, 48, 2, 189, 5, 0, 8, 3, 80, 2, 4, 2, 39, 1, 0, 6, 6, 12, None, 4, 0, 6691, 12, 9, 62, 2, 1, 0, 3, 2, 7, 1, 10, None, None, 624, 0, 10, 16, 0, None, 23, 37, 2, 0, 18, 1, 15, 44, 477, 4, 19, 58, 1, 57, 57, 25, 1, 0, 26, 1, None, 7, 61, 11, 40, 3, 1, 12, 0, 8, 2, 24, 5, 1, 218, 52, 20, 0, 14, 370, 3, 1, 1, 27, 2, 0, 2, 2, None, 4, 0, 471, 23, 1, 55, 1, None, 22, 5, 38, 0, 0, None, 1, 11, 3, 2, 6, None, 2, 0, 73, 5, 0, 0, 152, 0, 2, 11, 1, 1, 20, None, 24, 0, 3, 3, 8, 7, 0, 19, 2, 1, None, 25, 0, 12, 253, 2, 2, 0, None, None, 7, None, 0, 3, None, 4, 0, 3, 6, None, 4, 31, 9, None, 45, None, 8, 0, 1, 0, 0, 4, 2, 1, 0, 14, 485, 92, 0, 2, 0, 1, 1, 14, 0, 6, 5, 47, 1, 2, 0, 3, 12, 7, None, 4, None, 2, None, 3, 0, 9, None, 1, 1, 0, 12, 0, 0, 8, 8, 348, 10, 12, 9, 0, 0, 0, 3, None, 1, 3, 4, 6, 0, 9, 5, 14, 160, 2, 14, 3, None, 1, 1, 2, 1, 10, 1, 47, 

In [6]:
MQDict = {}
for i in range(comm_N):
    if (MQData[i] is not None) and (MQData[i] > 1e4):
        MQDict[start_comm_id + i] = MQData[i]
print(MQDict)

{21025608: 17001, 21027511: 20004, 21028971: 12558, 21029816: 41895, 21030237: 120258, 21030380: 14345, 21030451: 11367, 21030496: 32796, 21030653: 20379, 21030850: 18438, 21031029: 23312, 21031410: 10700, 21031589: 12376, 21031738: 13841, 21031741: 21924, 21032493: 53325, 21033476: 22458, 21034152: 111768, 21034326: 11415, 21034885: 17837, 21035285: 38881, 21035513: 154862, 21035520: 94418, 21038497: 17469, 21038657: 10904, 21041408: 55189, 21041492: 10396, 21041966: 34699, 21043710: 12683, 21044057: 30399, 21045875: 10524, 21046129: 11506, 21046577: 23800, 21048921: 16717, 21049097: 12872, 21054010: 10240, 21054043: 11258, 21054448: 25736, 21055197: 10181, 21055540: 18534, 21057565: 17182, 21059933: 17363, 21060187: 209767, 21060863: 10064, 21062063: 17341, 21062564: 22592, 21062677: 20090, 21062987: 10967, 21063186: 18289, 21063552: 15379, 21064118: 19073, 21064784: 11180, 21065761: 10038, 21066074: 18637, 21069499: 16110, 21071083: 11315, 21072801: 10978, 21073435: 15392, 21073592:

In [7]:
vk_session = vk_api.VkApi(login=settings.MY_USER_LOGIN, password=settings.MY_USER_PASSWORD)
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
vk = vk_session.get_api()

MQDict = {id : memb for id, memb in MQDict.items() if isCommunityAlive(id) is "Alive"}
print(MQDict)

{21025608: 17001, 21030237: 120258, 21030380: 14345, 21030451: 11367, 21030850: 18438, 21031029: 23312, 21034152: 111768, 21034326: 11415, 21034885: 17837, 21035285: 38881, 21035513: 154862, 21038497: 17469, 21041492: 10396, 21041966: 34699, 21044057: 30399, 21045875: 10524, 21046129: 11506, 21049097: 12872, 21054010: 10240, 21062564: 22592, 21064118: 19073, 21064784: 11180, 21065761: 10038, 21066074: 18637, 21069499: 16110, 21079287: 20685, 21081773: 12748, 21082539: 1872814, 21089879: 17350, 21090314: 609816, 21092922: 209375, 21093147: 20181, 21096399: 39307, 21097030: 16222, 21097353: 187761, 21099247: 11807, 21100008: 18792, 21100181: 13820, 21102217: 38523, 21102748: 18146, 21104755: 18283, 21105072: 25257, 21106090: 10997, 21109954: 98518, 21115588: 17137, 21115965: 15754, 21117022: 12328, 21118635: 795509, 21120591: 10150, 21121210: 35699, 21123456: 14237, 21125731: 128305, 21127508: 10249, 21131372: 17622, 21136525: 41307, 21137179: 23859, 21137336: 38528, 21137824: 10865, 211

In [8]:
IDDict = {}
for id, quan in MQDict.items():
    IDDict[id] = []
    for offset in range(0, quan, 25000):
        IDDict[id].append(vk_func_getMembIDList(vk, x=id, y=offset))
        
for id, ilist in IDDict.items():
    IDDict[id] = [z for x in IDDict[id] for y in x for z in y]

In [12]:
np.save(f'./data/{start_comm_id}to{start_comm_id + comm_N - 1}IDDict.npy', IDDict) 

In [11]:
for key, val in IDDict.items():
    print(f"{key}: {val[0:1000]}")

21025608: [2673, 9011, 18440, 38996, 39409, 40426, 98499, 98663, 98795, 98860, 98902, 99227, 117751, 122152, 154254, 171773, 266348, 269829, 294626, 327315, 339723, 371871, 375750, 402011, 410986, 454924, 519774, 570911, 574035, 582619, 589828, 595989, 614909, 660404, 722015, 766840, 802248, 872829, 892705, 951383, 995849, 1050183, 1153413, 1228175, 1317891, 1320100, 1325653, 1376770, 1378851, 1515555, 1550482, 1674670, 1681041, 1752111, 1768163, 1781578, 1783372, 1817986, 1827133, 1888828, 1890167, 1932323, 1939315, 1994483, 2018321, 2032364, 2058777, 2059332, 2068863, 2080540, 2094807, 2098115, 2116488, 2122541, 2131104, 2132577, 2165791, 2170782, 2178555, 2193287, 2198933, 2207537, 2223580, 2229204, 2246038, 2249699, 2370127, 2430524, 2437293, 2452436, 2458938, 2470084, 2481130, 2488142, 2540010, 2542219, 2568013, 2616889, 2620656, 2652716, 2759580, 2776246, 2777411, 2787274, 2800121, 2815865, 2824162, 2840878, 2862247, 2873192, 2987933, 3070492, 3087149, 3120875, 3121201, 3153543, 